In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Teste") \
    .master("local[*]")\
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
    .config("hive.exe.dynamic.partition.mode", "nonstrict") \
    .config("hive.metastore.uris", "thrift://localhost:9083") \
    .enableHiveSupport() \
    .getOrCreate()
    #spark://localhost:7077
    #.master("local[*]")\
    # .config("spark.dynamicAllocation.enabled", "true") \
    # .config("spark.shuffle.service.enabled", "true") \
    # .config("spark.executor.memory","512m") \
    # .config("spark.executor.cores","2") \

In [22]:
lines = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "reclamacoes") \
    .load()

In [23]:
lines.show()

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[5B 28 27 43 65 6...|reclamacoes|        0|     0|2024-10-15 16:20:...|            0|
|null|[5B 28 27 43 65 6...|reclamacoes|        0|     1|2024-10-15 16:41:...|            0|
+----+--------------------+-----------+---------+------+--------------------+-------------+



In [24]:
teste = lines.selectExpr("CAST(value AS STRING)").first()[-1]

In [25]:
relatos = eval(teste)

In [28]:
Schema = StructType() \
	.add('nomeEmpresa', 'string') \
    .add('status', 'string') \
    .add('tempoResposta', 'string') \
    .add('dataOcorrido', 'string') \
    .add('Cidade', 'string') \
    .add('UF', 'string') \
    .add('Relato', 'string') \
    .add('Resposta', 'string') \
    .add('Nota', 'string') \
    .add('Comentario', 'string')

In [29]:
df = spark.createDataFrame(relatos, schema=Schema)

In [30]:
df.show(50)

+--------------------+-------------+-------------+------------+--------------------+---+--------------------+--------------------+------+--------------------+
|         nomeEmpresa|       status|tempoResposta|dataOcorrido|              Cidade| UF|              Relato|            Resposta|  Nota|          Comentario|
+--------------------+-------------+-------------+------------+--------------------+---+--------------------+--------------------+------+--------------------+
|               Cemig|    Resolvido| no mesmo dia|  14/10/2024|            Contagem| MG|Qual o prazo para...|Olá, Felipe! Boa ...|Nota 5|<não há comentári...|
|               Cemig|    Resolvido| no mesmo dia|  14/10/2024|      Belo Horizonte| MG|não concordo com ...|Olá, Ana! Boa Tar...|Nota 5|      muito obrigada|
|               Cemig|    Resolvido| no mesmo dia|  14/10/2024|      Belo Horizonte| MG|Sou obrigado a pa...|Olá, Ana! Boa Tar...|Nota 5|<não há comentári...|
|               Cemig|    Resolvido| no mesmo 

In [31]:
df.count()

80

In [ ]:
query = teste.writeStream \
            .format("parquet") \
            .option("path", "hdfs://localhost:9000/data/teste") \
            .option("checkpointLocation", "hdfs://localhost:9000/tmp/checkpoint") \
            .start()

In [ ]:
query.awaitTermination()